<a href="https://colab.research.google.com/github/Petint/AI-and-DS/blob/main/ai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-2

In [ ]:
# Import all necessary
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Connect to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dara prep

In [ ]:
path = '/content/drive/MyDrive/'
input_shape = (96, 96, 3)
x = []
y = []

for folder in os.listdir(path):
  for images in os.listdir(os.path.join(path, folder)):
    imagepath = os.path.join(path, folder, images)
    image = cv.imread(imagepath)
    image = cv.resize(image, (input_shape[1], input_shape[0]))
    label = folder.replace('_', ' ')
    x.append(image)
    y.append(label)